This notebook is used to transform the newgroups files into one csv dataset.<br>
The features(columns) are the words in the vocabulary.<br>
The samples are the files. <br>
The target feature(last column) is the newsgroup(class) to which each file belongs to. <br>

This notebook uses some of the functions provided in the naive bayes squeleton.<br>
Due the the big amount of data this was ran on google colab resulting in a 1.3Gb csv file for training
and 900mb file for testing.

In [1]:

import pandas as pd
from google.colab import drive
import os
import pprint
import re
import codecs
import math
import itertools
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

ModuleNotFoundError: No module named 'google'

In [2]:
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
base_path_newsgroups = "/content/gdrive/MyDrive/newsgroups/"
path_newsgroups_train = os.path.join(base_path_newsgroups, "20news-bydate-train")
path_newsgroups_test = os.path.join(base_path_newsgroups, "20news-bydate-test")

In [4]:
subdirectory_names = [file_name for file_name in os.listdir(path_newsgroups_train)]

In [5]:
categories = sorted(subdirectory_names)
samples_per_cat = [len(list(os.listdir(os.path.join(path_newsgroups_train, cat_name)))) for cat_name in categories]


In [6]:
file_nbr=sum(samples_per_cat)
print('Total number of files :',file_nbr)

Total number of files : 7532


In [7]:
def tokenize_str(doc):
    return re.findall(r'\b\w\w+\b', doc) # return all words with #characters > 1


In [8]:
def tokenize_file(doc_file):
    with codecs.open(doc_file, encoding='latin1') as doc:
        doc = doc.read().lower()
        _header, _blankline, body = doc.partition('\n\n')
        return tokenize_str(body) # return all words with #characters > 1


In [10]:
min_count = 1
vocabulary = {}
num_docs = 0
classes = {}
priors = {}
conditionals = {}

num_docs = 0
for class_name in os.listdir(path_newsgroups_train):
    classes[class_name] = {"doc_counts": 0, "term_counts": 0, "terms": {}}
    path_class = os.path.join(path_newsgroups_train, class_name)
    for doc_name in os.listdir(path_class):
        terms = tokenize_file(os.path.join(path_class, doc_name))
        num_docs += 1
        classes[class_name]["doc_counts"] += 1

        # build vocabulary and count terms
        for term in terms:
            classes[class_name]["term_counts"] += 1
            if not term in vocabulary:
                vocabulary[term] = 1
                classes[class_name]["terms"][term] = 1
            else:
                vocabulary[term] += 1
                if not term in classes[class_name]["terms"]:
                    classes[class_name]["terms"][term] = 1
                else:
                    classes[class_name]["terms"][term] += 1

# remove terms with frequency < min_count
vocabulary = {k:v for k,v in vocabulary.items() if v > min_count}
Vocab_len=len(vocabulary)
print("Vocabulary size:",Vocab_len )


Vocabulary size: 60738


In [11]:
data=np.zeros((file_nbr,Vocab_len),int)
classification=[]
column_names=list(vocabulary.keys())

In [12]:
data.shape

(7532, 60738)

In [13]:
def transform(path,data,column_names,classification):
    i=0
    for class_name in os.listdir(path):
        path_class = os.path.join(path, class_name)
        for doc_name in os.listdir(path_class):
            terms =tokenize_file(os.path.join(path_class, doc_name))
            for j in range(data.shape[1]-1):
                if column_names[j] in terms:
                    data[i,j]=data[i,j]+1                     
            classification.append(class_name)
            i=i+1

                

In [14]:
transform(path_newsgroups_train,data,column_names,classification)

In [15]:

df_train=pd.DataFrame(data)
df_train.columns=column_names
df_train['Classification']=classification


In [16]:
df_train.to_csv('df_train.csv')
files.download('df_train.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>